In [ ]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
encoding= 'utf-8-sig'

nltk.download('stopwords')
stopwords = stopwords.words('english')
#stopwords = nltk.corpus.stopwords.words('english')

#加入新的停用詞
new_stopwords = ["Inc", "Co", "Ltd", "LLC","L.C.C","Llc", "Limited","Corporation","Corp","GmbH"]
stopwords.extend(new_stopwords)

database = pd.read_excel("./database.xlsx",sheet_name="工作表1")

## 清洗資料庫

In [ ]:
database["Assignee_Name_N"]=database["Assignee_Name"].str.title() #第一個字大小
database["Assignee_Name_N"]=database["Assignee_Name_N"].str.replace('[^A-Za-z0-9]+',' ',regex=True) #去標點符號
database["Assignee_Name_N"] = database["Assignee_Name_N"].apply(lambda words:' '.join(word.title() for word in words.split() if word not in stopwords)) #去停用詞
database["AC_N"]=database["AC"].str.title() #第一個字大小
database["AC_N"]=database["AC_N"].str.replace('[^A-Za-z0-9]+',' ',regex=True) #去標點符號
database["AC_N"] = database["AC_N"].apply(lambda words:' '.join(word.title() for word in words.split() if word not in stopwords)) #去停用詞

In [ ]:
database = database[["Assignee_Name_N","AC_N","AC"]]

## 清洗判別資料 

In [ ]:
df["Assignee_ORGNAME_N"]=df["Assignee_ORGNAME"].str.title() #第一個字大小
df["Assignee_ORGNAME_N"]=df["Assignee_ORGNAME_N"].str.replace('[^A-Za-z0-9]+',' ',regex=True) #去標點符號
df["Assignee_ORGNAME_N"] = df["Assignee_ORGNAME_N"].apply(lambda words:' '.join(word.title() for word in words.split() if word not in stopwords)) #去停用詞

## 字串對比

In [ ]:
from difflib import SequenceMatcher
import pandas as pd

s=0
f=0
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()


for i in range(0,len(df["Assignee_ORGNAME_N"])):
    label = df["Assignee_ORGNAME_N"][i]

    if len(database[database.Assignee_Name_N== label])>=1:
        s+=1 
        df['AC'][i] = database['AC'][database[database.Assignee_Name_N == label].index[0]]
        
    elif len(database[database.AC_N== label])>=1:
        df['AC'][i] = database['AC'][database[database.AC_N == label].index[0]]  
    elif label=='NULL':
        s+=1
        df['AC'][i]='NULL'
    else:
        for j in range(0,len(database["AC_N"])):
            similar = similarity(label,database["AC_N"][j])
            if similar >= 0.95:                
                df['AC'][i] = database['AC'][database[database.AC_N == label].index[0]]
            else:
                new_df = pd.DataFrame({
                "AC":[df["Assignee_ORGNAME"][i]],
                "Assignee_Name_N":[label],
                "AC_N":[label]})
                database.append(new_df)
                #database.to_csv("updateDatabase.csv",encoding="utf-8-sig")

#儲存檔案 比對資料 前後
df.to_csv("result.csv",encoding='utf-8-sig')
        
print("成功",s,"失敗",f)